June 2023


Martin Aaskov Karlsen  
maak@chem.au.dk  
maak-sdu @ GitHub  
Postdoc  
Ravnsbæk Group  
Aarhus University

# Merging .tif files from *operando* total scattering

This code is written for merging subframes from *operando* total scattering 
experiments conducted on beamline P02.1, PETRA III, DESY, using the 
`ravnsbaek.py` macro.

During the total scattering experiment, a single exposure exposure is split into 
multiple subframes, which are to be merged by this code.

At earlier beamtimes, an exposure time of four minutes per cycle was used, 
meaning that four subframes were to be merged. However, as long as the naming 
scheme below is followed, the code should work for other exposure times, number 
of frames to merge, etc.

## Naming scheme
For the code to be able to sort the files properly, the following name scheme is
expected:
```
POSITION_NAME_CYCLENUMBER-RUNNINGNUMBER.tif
```
e.g.:
```
N1_Na_AOD42_p11_0-00162.tif
```
where:
```
POSITION: N1  
NAME: Na_AOD42_p11  
CYCLEBUMBER: 0  
RUNNINGNUMBER: 00162
```
NB: Please use an underscore ('_') to separate the position from the sample 
'name.  
Please don't use a hyphen/dash ('-') in the filename except for separating
the cycle number and the running number.

## Running the code
The code should be run from a directory containing the `.tif` files that are to
be merged.

Only three modules are used in the code. The ``pathlib`` module is a part of the
standard library and comes with your Python installation. In addition, the 
``numpy`` and ``scikit-image`` modules are used. For the code to function, these 
modules need to be installed if not already.

If you are using a conda distribution and you need a new environment using e.g.
Python 3.11, you can create such one:
```
conda create -n py311 -c conda-forge python=3.11 numpy scikit-image jupyterlab
```
The newly created conda environment can then be activated:
```
conda activate py311
```
Then, the code can be run through Jupyter (lab or notebook)
```
jupyter-lab
```
or
```
jupyter-notebook
```
Alternatively, the code can be run directly from the command line:
from the command line:
```
ipython petra_p021_merge_tifs_operando.ipynb
```
Good luck and happy merging!


Imports.

In [ ]:
from pathlib import Path
import numpy as np
import skimage

Loading `.tif` files from current working directory, where `.tif` files
containing "dark" in the file name are excluded, as these are not to merged.

In [ ]:
tif_files_all = list(Path.cwd().glob("*.tif"))
tif_files = [tif for tif in tif_files_all if not "dark" in tif.name]

Sorting ``.tif`` files to be merged by sample position, cycle number, and 
running number in a dictionary.

In [ ]:
d = {}
for tif in tif_files:
    filename_split = tif.stem.split("_")
    position_name = "_".join(filename_split[0:-1])
    numbers = filename_split[-1].split("-")
    cycle, number = int(numbers[0]), int(numbers[1])
    if not position_name in d.keys():
        d[position_name] = {}
    if not cycle in d[position_name].keys():
        d[position_name][cycle] = {number: tif}
    else:
        d[position_name][cycle][number] = tif
for k in d.keys():
    d[k] = dict(sorted(d[k].items()))

Creating the following output paths in the current working direcotory for merged 
files if not already existing:
- tif_sum
- tif_avg
- npy_sum
- npy_avg

In [ ]:
tif_sum_path = Path.cwd() / "tif_sum"
tif_avg_path = Path.cwd() / "tif_avg"
npy_sum_path = Path.cwd() / "npy_sum"
npy_avg_path = Path.cwd() / "npy_avg"
d_output_paths = dict(tif_sum=tif_sum_path, 
                      tif_avg=tif_avg_path, 
                      npy_sum=npy_sum_path, 
                      npy_avg=npy_avg_path,
                      )
for p in d_output_paths.values():
    if not p.exists():
        p.mkdir()
for position_name in d.keys():
    for p in d_output_paths.values():
        if not (p / position_name).exists():
            (p / position_name).mkdir()

Function to merge ``.tif`` files both by summation and averaging. The merged 
files with be written in both ``.tif`` and ``.npy`` format to the output folders
created above.

In [26]:
def merge(position_name, cycle, d, d_paths):
    subframes = d.keys()
    for i, subframe in enumerate(subframes):
        print(f"\t\t{subframe}")
        file_path = d[subframe]
        if i == 0:
            array = np.array(skimage.io.imread(file_path))
        else:
            array = np.dstack((array, 
                               np.array(skimage.io.imread(file_path)),
                               ))
    array_sum, array_avg = np.sum(array, axis=-1), np.mean(array, axis=-1)
    subframes_str = '-'.join(str(subframe) for subframe in subframes)
    outname = f"{position_name}_{cycle}_{subframes_str}"
    np.save(d_paths["npy_sum"] / position_name / f"{outname}_sum", array_sum)
    np.save(d_paths["npy_avg"] / position_name / f"{outname}_avg", array_avg)
    skimage.io.imsave(d_paths["tif_sum"] / position_name / f"{outname}_sum.tif",
                      array_sum,
                      )
    skimage.io.imsave(d_paths["tif_avg"] / position_name / f"{outname}_avg.tif",
                      array_avg,
                      )

    return None

Merging `.tif` files both by summation and averaging and saving them to the
output folder from above.

In [ ]:
for position_name in d.keys():
    print(f"{80*'-'}\nMerging .tif files for: {position_name}")
    cycles = d[position_name].keys()
    for cycle in cycles:
        print(f"\tcycle: {cycle}")    
        merge(position_name, cycle, d[position_name][cycle], d_output_paths)
output_folders = [k for k in d_output_paths.keys()]
print(f"{80*'-'}\nDone merging files. Please the {output_folders} folders.")